In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 16

class Autoencoder(nn.Module):
    def __init__(self, input_size=128):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm2 = nn.BatchNorm2d(channels_base)
        self.batchnorm3 = nn.BatchNorm2d(channels_base * 2)
        
        self.leaky_relu1 = nn.LeakyReLU(0.1)
        self.avgpool1 = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        self.upscale1 = nn.Upsample(scale_factor=scale_factor)

    def forward(self, input):
        # encoder
        x = self.leaky_relu1(self.batchnorm2(self.conv1(input)))
        x = self.avgpool1(x)
        x = self.leaky_relu1(self.batchnorm3(self.conv2(x)))
        x = self.avgpool1(x)
        
        # decoder
        x = self.leaky_relu1(self.batchnorm2(self.convtrans1(x)))
        x = self.upscale1(x)
        x = self.leaky_relu1(self.batchnorm1(self.convtrans2(x)))
        x = self.upscale1(self.convtrans3(x))

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             160
       BatchNorm2d-2           [-1, 16, 32, 32]              32
         LeakyReLU-3           [-1, 16, 32, 32]               0
         AvgPool2d-4           [-1, 16, 16, 16]               0
            Conv2d-5           [-1, 32, 16, 16]           4,640
       BatchNorm2d-6           [-1, 32, 16, 16]              64
         LeakyReLU-7           [-1, 32, 16, 16]               0
         AvgPool2d-8             [-1, 32, 8, 8]               0
   ConvTranspose2d-9             [-1, 16, 8, 8]           4,624
      BatchNorm2d-10             [-1, 16, 8, 8]              32
        LeakyReLU-11             [-1, 16, 8, 8]               0
         Upsample-12           [-1, 16, 16, 16]               0
  ConvTranspose2d-13            [-1, 8, 16, 16]           1,160
      BatchNorm2d-14            [-1, 8,

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00360559 (0.01423273), PSNR 24.43024063 (22.40936665), SSIM 0.63933557 (0.52827741)
Finished training epoch 0
Validate: MSE 0.00370152 (0.00381052), PSNR 24.31619835 (24.30093351), SSIM 0.68040329 (0.74871385)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00296330 (0.00332527), PSNR 25.28224373 (24.81851433), SSIM 0.73149073 (0.66916819)
Finished training epoch 1
Validate: MSE 0.00397620 (0.00318821), PSNR 24.00531578 (25.01810208), SSIM 0.68614525 (0.75658045)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00287625 (0.00300735), PSNR 25.41172791 (25.24339021), SSIM 0.71392167 (0.71050670)
Finished training epoch 2
Validate: MSE 0.00587287 (0.00448388), PSNR 22.31149673 (23.56459664), SSIM 0.68682438 (0.75580262)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00289193 (0.00286946), PSNR 25.38811874 (25.44579466), SSIM 0.73539162 (0.73083740)
Finished training epoch 3
Validate: MSE 0.00755179 (0.0

Epoch: 30, MSE 0.00227334 (0.00237477), PSNR 26.43336296 (26.26674518), SSIM 0.77707118 (0.77748573)
Finished training epoch 30
Validate: MSE 0.00812241 (0.00665191), PSNR 20.90314865 (21.88042761), SSIM 0.66998971 (0.74670695)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00276641 (0.00237060), PSNR 25.58083725 (26.27603130), SSIM 0.75964379 (0.77743830)
Finished training epoch 31
Validate: MSE 0.00673420 (0.00541941), PSNR 21.71713638 (22.79689595), SSIM 0.66648555 (0.74728371)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00234353 (0.00237140), PSNR 26.30128670 (26.27248810), SSIM 0.78803188 (0.77735672)
Finished training epoch 32
Validate: MSE 0.00960072 (0.00774628), PSNR 20.17695999 (21.21287620), SSIM 0.66114974 (0.73960072)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00242603 (0.00236479), PSNR 26.15103531 (26.28541195), SSIM 0.76140726 (0.77751473)
Finished training epoch 33
Validate: MSE 0.00666616 (0.00505192), PSNR 

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return func(*args, **kwargs)


Validate: MSE 0.00437062 (0.00447301), PSNR 23.59457207 (23.59702414), SSIM 0.68027258 (0.73351757)
Finished validation.
Starting training epoch 57
Epoch: 57, MSE 0.00226281 (0.00232472), PSNR 26.45351791 (26.36048610), SSIM 0.77816999 (0.77813921)
Finished training epoch 57
Validate: MSE 0.00374006 (0.00324350), PSNR 24.27120972 (24.94075464), SSIM 0.68715107 (0.75208616)
Finished validation.
Starting training epoch 58
Epoch: 58, MSE 0.00231987 (0.00232442), PSNR 26.34535599 (26.36003751), SSIM 0.77581531 (0.77803668)
Finished training epoch 58
Validate: MSE 0.00318860 (0.00283497), PSNR 24.96400261 (25.54520769), SSIM 0.70411396 (0.77235930)
Finished validation.
Starting training epoch 59
Epoch: 59, MSE 0.00305285 (0.00232346), PSNR 25.15294266 (26.36162136), SSIM 0.74818933 (0.77804823)
Finished training epoch 59
Validate: MSE 0.00520017 (0.00431461), PSNR 22.83982277 (23.76097560), SSIM 0.68889558 (0.75924768)
Finished validation.
Starting training epoch 60
Epoch: 60, MSE 0.0024223

Validate: MSE 0.01423095 (0.01233459), PSNR 18.46765900 (19.16394115), SSIM 0.61909580 (0.69593233)
Finished validation.
Starting training epoch 87
Epoch: 87, MSE 0.00263878 (0.00230401), PSNR 25.78596497 (26.40142365), SSIM 0.78110379 (0.77851384)
Finished training epoch 87
Validate: MSE 0.00974762 (0.00804462), PSNR 20.11101532 (21.05094269), SSIM 0.65757489 (0.73639534)
Finished validation.
Starting training epoch 88
Epoch: 88, MSE 0.00239710 (0.00229723), PSNR 26.20314217 (26.41038283), SSIM 0.75898647 (0.77849831)
Finished training epoch 88
Validate: MSE 0.01228621 (0.00989955), PSNR 19.10581970 (20.12877969), SSIM 0.62941408 (0.70830421)
Finished validation.
Starting training epoch 89
Epoch: 89, MSE 0.00245612 (0.00230184), PSNR 26.09751129 (26.40486204), SSIM 0.77399701 (0.77837149)
Finished training epoch 89
Validate: MSE 0.01547927 (0.01300450), PSNR 18.10249329 (18.92906790), SSIM 0.61372834 (0.69324462)
Finished validation.
Starting training epoch 90
Epoch: 90, MSE 0.0030189

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.02601890 (0.02225824), PSNR 15.84710979 (16.57728632), SSIM 0.55928606 (0.64061785)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()